<a href="https://colab.research.google.com/github/Anthei0774/Advent-of-Code-2021/blob/main/Day_18_Snailfish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# https://adventofcode.com/2021/day/18

# Part 1

In [ ]:
def explode(line, verbose=False):

    ll = len(line)
    new_line = line

    # search for exploding segments
    mid_start = 0
    while mid_start < ll:
        prefix = line[: mid_start + 1]
        if (prefix.count('[') - prefix.count(']') == 5):
            break
        mid_start += 1

    if mid_start != ll:

        # add end position to segment
        mid_end = line.find(']', mid_start)
    
        mid = line[mid_start : mid_end + 1]
        if verbose: print('Exploding:', mid)

        # searching for the first digit on the left
        left_end = mid_start - 1
        while line[left_end] in ['[', ']', ',']:
            left_end -= 1
            if left_end == -1:
                break
        # if left_end != -1: print('Left:', line[: left_end + 1])

        # same search on the right
        right_start = mid_end + 1
        while line[right_start] in ['[', ']', ',']:
            right_start += 1
            if right_start == ll:
                break
        # if right_start != ll: print('Right:', line[right_start - 1 :])
            
        # restructuring the line
        new_line = ''

        # left side
        if left_end != -1:

            # see if a 2 digit number found on the left
            left_digit2 = (line[left_end - 1] not in ['[', ']', ','])
            new_line += line[: left_end - left_digit2]

            n = int(line[left_end - left_digit2 : left_end + 1])

            # see if a 2 digit number found on the middle-left
            mid_digit2 = (mid[2] not in ['[', ']', ','])
            n += int(mid[1 : 2 + mid_digit2])

            new_line += str(n)

            # add text in between 
            new_line += line[left_end + 1 : mid_start]
                
        else:
            new_line += line[: mid_start]

        # middle segment
        new_line += '0'

        # right side
        if right_start != ll:
            new_line += line[mid_end + 1 : right_start]

            right_digit2 = (line[right_start + 1] not in ['[', ']', ','])
            n = int(line[right_start : right_start + right_digit2 + 1])

            # see if a 2 digit number found on the middle-left
            comma_pos = mid.find(',')
            mid_digit2 = (mid[comma_pos + 2] not in ['[', ']', ','])
            n += int(mid[comma_pos + 1 : comma_pos + 2 + mid_digit2])

            new_line += str(n)

            # remaining text between
            new_line += line[right_start + right_digit2 + 1 :]

        else:
            new_line += line[mid_end + 1 :]

    return new_line

################################################################################

def split(line, verbose=False):

    # print('Line:', line)
    ll = len(line)
    new_line = line

    # search for split criteria
    i = 0
    while i < ll - 1:
        if (line[i] not in ['[', ']', ',']) and (line[i + 1] not in ['[', ']', ',']):
            break
        i += 1

    # building up a new line
    if i != ll - 1:
        if verbose: print('Splitting:', line[i : i + 2])

        # left
        new_line = line[: i]

        # middle
        n = int(line[i : i + 2])
        l = n // 2
        r = n - l
        new_line += ('[' + str(l) + ',' + str(r) + ']')

        # right
        new_line += line[i + 2 :]
        if verbose: print('Split:', new_line)

        # a split can generate an explosion - check
        new_line = explode(new_line)
        if verbose: print('Explode:', new_line, '\n')

    return new_line

In [ ]:
# test explode against input example
test = '''[[[[[9,8],1],2],3],4]
[7,[6,[5,[4,[3,2]]]]]
[[6,[5,[4,[3,2]]]],1]
[[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]
[[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]'''

test = test.split('\n')



for line in test:

    print('Line:', line)
    fin_line = line

    # explode until new line is generated
    while True:
        new_line = explode(fin_line)
        if fin_line == new_line:
            break

        fin_line = new_line
        print('New line:', fin_line)

    print('\n---\n')

In [ ]:
def add(line1, line2, verbose=False):

    fin_line = '[' + line1 + ',' + line2 + ']'
    if verbose: print('Addition:', fin_line, '\n')

    # explosion sweep through
    while True:
        new_line = explode(fin_line, verbose)
        if fin_line == new_line:
            break
        fin_line = new_line
        if verbose: print('Explode:', fin_line, '\n')

    # split / explosion sweep
    while True:
        new_line = split(fin_line, verbose)
        if fin_line == new_line:
            break
        fin_line = new_line

    return new_line



# check again, now addition
result = add('[[[[4,3],4],4],[7,[[8,4],9]]]', '[1,1]', True)

In [ ]:
# moar testing
test = '''[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]
[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]
[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]
[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]
[7,[5,[[3,8],[1,4]]]]
[[2,[2,2]],[8,[8,1]]]
[2,9]
[1,[[[9,3],9],[[9,0],[0,7]]]]
[[[5,[7,4]],7],1]
[[[[4,2],2],6],[8,7]]'''

test = test.split('\n')

fin_line = test[0]

for i in range(1, len(test)):
    fin_line = add(fin_line, test[i])
    print('Iter', i, '- line:', fin_line, '\n')

In [ ]:
def reduce(line):
    # print(line)

    # single number case
    if line.count(',') == 0:
        return line

    # single pair case
    if line.count(',') == 1:
        s = 0
        nums = line[1 : -1].split(',')
        nums = list(map(int, nums))
        s += nums[0] * 3
        s += nums[1] * 2
        s = str(s)

    # nested struct case
    if line.count(',') >= 2:

        # find middle separator comma
        i = line.find(',')
        prefix = line[: i]
        while prefix.count('[') - prefix.count(']') != 1:
            i = line.find(',', i + 1)
            prefix = line[: i]
        
        # reduce structure left and right
        left = reduce(line[1 : i])
        right = reduce(line[i + 1 : -1])
        s = '[' + left + ',' + right + ']'

    # print(s)
    return s

In [ ]:
# and some testing :)
lines = '''[[1,2],[[3,4],5]]
[[[[0,7],4],[[7,8],[6,0]]],[8,1]]
[[[[1,1],[2,2]],[3,3]],[4,4]]
[[[[3,0],[5,3]],[4,4]],[5,5]]
[[[[5,0],[7,4]],[5,5]],[6,6]]'''

lines = lines.split('\n')

for l in lines:

    fin_line = l
    c = fin_line.count(',')

    while c != 0:
        fin_line = reduce(fin_line)
        c = fin_line.count(',')

    print(l, fin_line)

In [ ]:
def do_homework(lines):
    lines = lines.split('\n')

    # sequentially adding them together
    fin_line = lines[0]
    for i in range(1, len(lines)):
        fin_line = add(fin_line, lines[i])
    print('Final snailfish number:', fin_line)

    c = fin_line.count(',')
    while c != 0:
        fin_line = reduce(fin_line)
        c = fin_line.count(',')

    print('Sum after reduction:', fin_line)

In [ ]:
# test against puzzle input
with open('18.txt', 'r') as f:
    puzzle_input = f.read()

do_homework(puzzle_input)

# Part 2

In [ ]:
# executing a simple quadratic search for the highest magnitude
def search(lines):
    M = 0
    M_i = -1
    M_j = -1

    # iterate with two indexek
    for i in range(len(lines)):
        for j in range(len(lines)):
            if i != j:

                # add lines together, calculate magnitude
                line = add(lines[i], lines[j])

                fin_line = line
                c = fin_line.count(',')
                while c != 0:
                    fin_line = reduce(fin_line)
                    c = fin_line.count(',')

                m = int(fin_line)

                # save max
                if M < m:
                    M = m
                    M_i = i
                    M_j = j

    return M, M_i, M_j

In [ ]:
# testing, yeaaaah
test = '''[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]'''

test = test.split('\n')

M, M_i, M_j = search(test)

print('Left:', test[M_i], '\n')

print('Right:', test[M_j], '\n')

print('Value:', M)

In [ ]:
# finally
test = puzzle_input.split('\n')

M, M_i, M_j = search(test)

print('Left:', test[M_i], '\n')

print('Right:', test[M_j], '\n')

print('Value:', M)